# Offcuts

Just a place to dump offcuts of code

In [161]:
class LSTMTagger(nn.Module):
    def __init__(self, vocab_size, n_hidden, n_fac, bs, nl):
        super().__init__()
        self.n_hidden = n_hidden
        self.vocab_size,self.nl = vocab_size,nl
        self.e = nn.Embedding(vocab_size, n_fac)
        self.rnn = nn.LSTM(n_fac, n_hidden, nl, dropout=0.5)
        self.l_out = nn.Linear(n_hidden, vocab_size)
        self.init_hidden(bs)
        
    def forward(self, cs):
        bs = cs[0].size(0)
        if self.h[0].size(1) != bs: self.init_hidden(bs)
        outp,h = self.rnn(self.e(cs), self.h)
#         self.h = repackage_var(h)
        return F.log_softmax(self.l_out(outp), dim=-1).view(-1, self.vocab_size)
    
    def init_hidden(self, bs):
        self.h = (Variable(torch.zeros(self.nl, bs, self.n_hidden)),
                  Variable(torch.zeros(self.nl, bs, self.n_hidden)))

In [162]:
batch_size = 1
use_cuda = False
criterion = nn.CrossEntropyLoss()
model = LSTMTagger(n_fac=50,n_hidden=256,vocab_size=len(note_to_int),bs=batch_size,nl=8)
if(use_cuda):
    model.cuda()
    criterion.cuda()
trainer = ModuleTrainer(model)
trainer.set_optimizer(optim.Adam, lr=1e-3)
trainer.set_loss(criterion)

# Bug in torchsample?
trainer._has_multiple_loss_fns = False

model

LSTMTagger(
  (e): Embedding(120, 50)
  (rnn): LSTM(50, 256, num_layers=8, dropout=0.5)
  (l_out): Linear(in_features=256, out_features=120, bias=True)
)

In [111]:
trainer.fit(tensor(x_tr), tensor(y_tr), num_epoch=4, batch_size=batch_size, shuffle=False)

Epoch 1/4:   0%|          | 1/721 [00:00<01:19,  9.09 batches/s]


ValueError: Expected input batch_size (256) to match target batch_size (32).

In [ ]:
trainer.fit(each_tensor(xs), each_tensor(ys), num_epoch=4, batch_size=batch_size, shuffle=False)